In [1]:
// Enable compiler to use Java classpath (REMOVED the invalid doc.value line)
interp.configureCompiler(c => {
c.settings.usejavacp.value = true
})

// Configure Coursier to fetch doc JARs
// Import Spark
import $ivy.`org.apache.spark::spark-sql:3.5.7`
// import Almond Spark plugin
// import $ivy.`sh.almond::almond-spark:0.14.0-RC8`

import org.apache.logging.log4j.{LogManager, Level}
import org.apache.logging.log4j.core.config.Configurator
// Set log levels BEFORE creating SparkSession
Configurator.setRootLevel(Level.WARN)
Configurator.setLevel("org.apache.spark", Level.WARN)
Configurator.setLevel("org.apache.spark.executor.Executor", Level.WARN)

import $ivy.$
import org.apache.logging.log4j.{LogManager, Level}
import org.apache.logging.log4j.core.config.Configurator

In [2]:
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}

// Silence logs
Logger.getLogger("org").setLevel(Level.ERROR)
Logger.getLogger("akka").setLevel(Level.ERROR)

val spark = {
  NotebookSparkSession.builder()
    .appName("2025-11-07-SparkDataset")
    .master("local[*]")
    .config("spark.driver.host", "localhost")
    .config("spark.driver.bindAddress", "0.0.0.0")
    .config("spark.driver.memory", "6g")
    .config("spark.ui.showConsoleProgress", "false")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .getOrCreate()
}

// ⭐ CRITICAL: This line forces the UI widget to persist after the cell is done
spark

spark.sparkContext.setLogLevel("ERROR")
import spark.implicits._

println(s"✅ Spark ${spark.version} ready")
println(s"🌐 Spark UI: http://localhost:4040")

Loading spark-stubs

Getting spark JARs

Creating SparkSession

07:25:26.671 [scala-kernel-interpreter-1] WARN  org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark UI

✅ Spark 3.5.7 ready
🌐 Spark UI: http://localhost:4040


import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
spark: SparkSession = org.apache.spark.sql.SparkSession@7c8da0db
res2_5: SparkSession = org.apache.spark.sql.SparkSession@7c8da0db
import spark.implicits._

In [3]:
// create a case class for the `nodes-entities.csv`
/**
  * Represents an entity from the nodes-entities.csv file.
  *
  * @param node_id The unique identifier for the node.
  * @param name The name of the entity.
  * @param original_name The original name of the entity, if different.
  * @param former_name A previous name for the entity.
  * @param jurisdiction The legal jurisdiction of the entity.
  * @param jurisdiction_description A description of the jurisdiction.
  * @param company_type The type of company (e.g., Limited, Corp).
  * @param address The registered address of the entity.
  * @param incorporation_date The date the entity was incorporated.
  * @param inactivation_date The date the entity was inactivated.
  * @param struck_off_date The date the entity was struck off the register.
  * @param status The current status of the entity.
  * @param service_provider The service provider associated with the entity.
  * @param ibc_ruc The IBC or RUC number.
  * @param country_codes The country codes associated with the entity.
  * @param countries The countries associated with the entity.
  * @param sourceID The ID of the data source.
  * @param valid_until The date until which the data is considered valid.
  * @param note Any additional notes.
  */
case class EntityNode(
    node_id: String,
    name: Option[String],
    original_name: Option[String],
    former_name: Option[String],
    jurisdiction: Option[String],
    jurisdiction_description: Option[String],
    company_type: Option[String],
    address: Option[String],
    incorporation_date: Option[String],
    inactivation_date: Option[String],
    struck_off_date: Option[String],
    status: Option[String],
    service_provider: Option[String],
    ibcRuc: Option[String],
    country_codes: Option[String],
    countries: Option[String],
    sourceID: Option[String],
    valid_until: Option[String],
    note: Option[String]
)


defined class EntityNode

In [4]:
// Read CSV and convert to Dataset
val entityNodeDS = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv("full-oldb.LATEST/nodes-entities.csv")
  .as[EntityNode]

// Example queries
entityNodeDS.show(4)

+--------+--------------------+--------------------+-----------+------------+------------------------+------------+--------------------+-----------+------------------+-----------------+---------------+---------+---------+----------------+------+-------------+---------+-------------+--------------------+----+
| node_id|                name|       original_name|former_name|jurisdiction|jurisdiction_description|company_type|             address|internal_id|incorporation_date|inactivation_date|struck_off_date|dorm_date|   status|service_provider|ibcRUC|country_codes|countries|     sourceID|         valid_until|note|
+--------+--------------------+--------------------+-----------+------------+------------------------+------------+--------------------+-----------+------------------+-----------------+---------------+---------+---------+----------------+------+-------------+---------+-------------+--------------------+----+
|10000001|TIANSHENG INDUSTR...|TIANSHENG INDUSTR...|       NULL|      

entityNodeDS: Dataset[EntityNode] = [node_id: string, name: string ... 19 more fields]

In [5]:
// create a case class for the `nodes-intermediaries.csv`
/**
  * Represents an intermediary from the nodes-intermediaries.csv file.
  *
  * @param node_id The unique identifier for the node.
  * @param name The name of the intermediary.
  * @param status The current status of the intermediary.
  * @param internal_id An internal identifier used by the source.
  * @param address The address of the intermediary.
  * @param countries The countries associated with the intermediary.
  * @param country_codes The country codes associated with the intermediary.
  * @param sourceID The ID of the data source.
  * @param valid_until The date until which the data is considered valid.
  * @param note Any additional notes.
  */
case class Intermediary(
    node_id: String,
    name: String,
    status: Option[String],
    internal_id: Option[String],
    address: Option[String],
    countries: Option[String],
    country_codes: Option[String],
    sourceID: String,
    valid_until: String,
    note: Option[String]
)

defined class Intermediary

In [6]:
// Read CSV and convert to Dataset
val intermediaryDS = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv("full-oldb.LATEST/nodes-intermediaries.csv")
  .as[Intermediary]

// Example queries
intermediaryDS.show(4, truncate=false)

+--------+--------------------------+---------+-----------+-----------------------------------------------------------------------------------------------+-------------+-------------+-------------+-----------------------------------------------+----+
|node_id |name                      |status   |internal_id|address                                                                                        |countries    |country_codes|sourceID     |valid_until                                    |note|
+--------+--------------------------+---------+-----------+-----------------------------------------------------------------------------------------------+-------------+-------------+-------------+-----------------------------------------------+----+
|11000001|MICHAEL PAPAGEORGE, MR.   |ACTIVE   |10001      |MICHAEL PAPAGEORGE; MR. 106 NICHOLSON STREET BROOKLYN PRETORIA 0002; GAUTENG (PWV) SOUTH AFRICA|South Africa |ZAF          |Panama Papers|The Panama Papers  data is current through 2015|NU

intermediaryDS: Dataset[Intermediary] = [node_id: string, name: string ... 8 more fields]

In [7]:
// create a case class for the `nodes-intermediaries.csv`
/**
  * Represents a node from the nodes-others.csv file.
  *
  * @param node_id The unique identifier for the node.
  * @param name The name associated with the node.
  * @param `type` The type of the node (e.g., 'company', 'person').
  * @param country_codes The country codes associated with the node.
  * @param countries The countries associated with the node.
  * @param sourceID The ID of the data source.
  * @param valid_until The date until which the data is considered valid.
  * @param note Any additional notes.
  */
case class OtherNode(
  node_id: String,
  name: Option[String],
  `type`: Option[String],
  country_codes: Option[String],
  countries: Option[String],
  sourceID: Option[String],
  valid_until: Option[String],
  note: Option[String]
)


defined class OtherNode

In [9]:
// Read CSV and convert to Dataset
val otherNodeDS = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv("full-oldb.LATEST/nodes-others.csv")
  .as[OtherNode]

// Example queries
otherNodeDS.show(4)

+--------+--------------------+--------------------+------------------+---------------+-----------+------------+------------------------+---------+-------------+--------------------+--------------------+--------------------+
| node_id|                name|                type|incorporation_date|struck_off_date|closed_date|jurisdiction|jurisdiction_description|countries|country_codes|            sourceID|         valid_until|                note|
+--------+--------------------+--------------------+------------------+---------------+-----------+------------+------------------------+---------+-------------+--------------------+--------------------+--------------------+
|85004929|ANTAM ENTERPRISES...|LIMITED LIABILITY...|       18-MAY-1983|           NULL|28-NOV-2012|          AW|                   Aruba|     NULL|         NULL|Paradise Papers -...|Aruba corporate r...|Closed date stand...|
|85008443|      DEVIATION N.V.|LIMITED LIABILITY...|       28-JUN-1989|    31-DEC-2002|       NULL| 

otherNodeDS: Dataset[OtherNode] = [node_id: int, name: string ... 11 more fields]

In [9]:
scala.util.Properties.versionNumberString

res9: String = "2.13.17"

In [10]:
spark.stop()